World population data cleaning and import

In [206]:
import pandas as pd
import numpy as np
import sys
world_population=pd.read_csv('Data/world_population.csv')
print(world_population.head(5))

  Country Name Country Code     Indicator Name Indicator Code       1960  \
0        Aruba          ABW  Population, total    SP.POP.TOTL    54211.0   
1  Afghanistan          AFG  Population, total    SP.POP.TOTL  8996351.0   
2       Angola          AGO  Population, total    SP.POP.TOTL  5643182.0   
3      Albania          ALB  Population, total    SP.POP.TOTL  1608800.0   
4      Andorra          AND  Population, total    SP.POP.TOTL    13411.0   

        1961       1962       1963       1964       1965  ...        2010  \
0    55438.0    56225.0    56695.0    57032.0    57360.0  ...    101669.0   
1  9166764.0  9345868.0  9533954.0  9731361.0  9938414.0  ...  28803167.0   
2  5753024.0  5866061.0  5980417.0  6093321.0  6203299.0  ...  23369131.0   
3  1659800.0  1711319.0  1762621.0  1814135.0  1864791.0  ...   2913021.0   
4    14375.0    15370.0    16412.0    17469.0    18549.0  ...     84449.0   

         2011        2012        2013        2014        2015        2016  \
0  

Unstacking year data

In [197]:
year=['1960'
,'1961'
,'1962'
,'1963'
,'1964'
,'1965'
,'1966'
,'1967'
,'1968'
,'1969'
,'1970'
,'1971'
,'1972'
,'1973'
,'1974'
,'1975'
,'1976'
,'1977'
,'1978'
,'1979'
,'1980'
,'1981'
,'1982'
,'1983'
,'1984'
,'1985'
,'1986'
,'1987'
,'1988'
,'1989'
,'1990'
,'1991'
,'1992'
,'1993'
,'1994'
,'1995'
,'1996'
,'1997'
,'1998'
,'1999'
,'2000'
,'2001'
,'2002'
,'2003'
,'2004'
,'2005'
,'2006'
,'2007'
,'2008'
,'2009'
,'2010'
,'2011'
,'2012'
,'2013'
,'2014'
,'2015'
,'2016'
,'2017']

In [198]:
world_population=pd.melt(world_population, id_vars='Country Name', value_vars=year)


In [199]:
#format column names and dtypes
world_population=world_population.rename(columns={'Country Name':'country','variable':'year','value':'population'})
print(world_population.head(5))

world_population.dtypes


       country  year  population
0        Aruba  1960     54211.0
1  Afghanistan  1960   8996351.0
2       Angola  1960   5643182.0
3      Albania  1960   1608800.0
4      Andorra  1960     13411.0


country        object
year           object
population    float64
dtype: object

In [200]:
world_population['year']=world_population['year'].astype(int)

world_population.dtypes

country        object
year            int64
population    float64
dtype: object

In [202]:
#clean missing values
world_population1=world_population.dropna(axis=0,how='any')
print(world_population1.isnull().sum())


country       0
year          0
population    0
dtype: int64


In [203]:
world_population1.to_csv(r'Data/world_population1.csv',index=False)

In [218]:
import psycopg2

conn = psycopg2.connect(database="TT", user="tt", password="123", host="127.0.0.1", port="5432")

cur=conn.cursor()
q1='CREATE TABLE world_population (country varchar(60),year integer,population float8);'


cur.execute(q1)
conn.commit()
conn.close()

In [219]:
#This function uploads csv to a target table
 

def pg_load_table(file_path, table_name, dbname, host, port, user, pwd):
    
    try:
        conn = psycopg2.connect(dbname=dbname, host=host, port=port,\
         user=user, password=pwd)
        print("Connecting to Database")
        cur = conn.cursor()
        f = open(file_path, "r")
        # Truncate the table first
        cur.execute("Truncate {} Cascade;".format(table_name))
        print("Truncated {}".format(table_name))
        # Load table from the file with header
        cur.copy_expert("copy {} from STDIN CSV HEADER QUOTE '\"'".format(table_name), f)
        cur.execute("commit;")
        print("Loaded data into {}".format(table_name))
        conn.close()
        print("DB connection closed.")

    except Exception as e:
        print("Error: {}".format(str(e)))
        

# Execution
file_path = 'Data/world_population1.csv'
table_name = 'world_population'
dbname = 'TT'
host = '127.0.0.1'
port = '5432'
user = 'tt'
pwd = '123'
pg_load_table(file_path, table_name, dbname, host, port, user, pwd)


Connecting to Database
Truncated world_population
Loaded data into world_population
DB connection closed.


In [220]:
conn = psycopg2.connect(database="TT", user="tt", password="123", host="127.0.0.1", port="5432")
cur=conn.cursor()
q2='SELECT * from world_population LIMIT 5;'
cur.execute(q2)
x=cur.fetchall()

In [221]:
print(x)

[('Aruba', 1960, 54211.0), ('Afghanistan', 1960, 8996351.0), ('Angola', 1960, 5643182.0), ('Albania', 1960, 1608800.0), ('Andorra', 1960, 13411.0)]
